## Introduction to Data Science

### Data Science Tasks: Classification and Filtering 

In [1]:
import os
import sys
import re
import math
import feedparser
import time
import string
import datetime
import sqlite3
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

Specifying the path to the files:

In [2]:
outputs = "../outputs/"

dbfile1 = "treino.sqlite"
dbfile2 = "treinoblogs.sqlite"
outblog = "blogoutputrss.xml"

db_teste = os.path.join(outputs,dbfile1)
db_blog = os.path.join(outputs,dbfile2)
rssblogoutput = os.path.join(outputs,outblog)

stopwords = nltk.corpus.stopwords.words('portuguese') + nltk.corpus.stopwords.words('english')

#### First block of functions: feature extraction:

In [3]:
def getwords(html):
    '''Remove the HTML tags and cleans the feeds files;splits the sentences 
    by the non alpha characters and converts all words to lowercase.
    Ignores bigger and too small words'''
    #splitter = re.compile('\\W*', flags=re.U)
    #splitter=re.compile(r'[^A-Z^a-z]+', flags=re.U)
    #words=[s.lower() for s in splitter.split(html) if len(s)>2 and len(s)<20]
    words = BeautifulSoup(html, "lxml").findAll(text=True)[0].split(' ')
    words = [w.strip(string.punctuation) for w in words if len(w)>2 and len(w)<20 and w not in stopwords]
    return dict([(w,1) for w in words])

In [4]:
def entryfeatures(entry):
    '''Used when our features are not documents, but feeds rss'''
    splitter=re.compile(r'\W*')
    f={}
    # Extract title words
    titlewords=[s.lower() for s in splitter.split(entry['title']) if len(s)>2 and len(s)<20 and s not in stopwords]
    for w in titlewords: f['Title:'+w]=1
    # Extract summary words
    summarywords=[s.lower() for s in splitter.split(entry['summary']) if len(s)>2 and len(s)<20 and s not in stopwords]
    # Count lowercase words
    uc=0
    for i in range(len(summarywords)):
        w=summarywords[i]
        f[w]=1
        if w.isupper(): uc+=1
        # Features are words in the feed summary
        if i < len(summarywords)-1:
            twowords=' '.join(summarywords[i:i+1])
            f[twowords]=1
    # Save publisher information
    f['Publisher:'+entry['publisher']]=1
    # Too many uppercase words are indicated
    if (float(uc)/(len(summarywords)+1))>0.3:
        f['MAIUSCULAS']=1
    return f

#### Second block of functions: classification

In [5]:
class classifier:
    ''' Represents the classifier, storing what's learnt from training.
    fc saves the combination words/categories {'word': {'bad': 3, 'good': 2}}
    and cc is a dictionary that stores the number of times a category was used
    {'bad': 3, 'good': 2}. Will be used when no DB is used.
    Getfeatures is the feature extraction function to be used'''
    def __init__(self, getfeatures, filename=None, usedb=False):
        self.fc={}
        self.cc={}
        self.getfeatures = getfeatures
        self.usedb = usedb
    
    def setdb(self,dbfile):
        '''When using a database and not dictionaries, to persist the information
        across sessions'''
        self.con = sqlite3.dbapi2.connect(dbfile)    
        self.con.execute('CREATE TABLE IF NOT EXISTS fc(feature,category,count)')
        self.con.execute('CREATE TABLE IF NOT EXISTS cc(category,count)')

    def fcount(self,f,cat):
        '''Returns the number of times a feature appears in a category'''
        if not self.usedb:
            if f in self.fc and cat in self.fc[f]: 
                return float(self.fc[f][cat])
            else: 
                return 0
        else:
            query = 'SELECT COUNT(*) FROM fc WHERE feature=? AND category=?'
            res = self.con.execute(query,(f,cat)).fetchone()
            if res == None: 
                return 0
            else: 
                return float(res[0])

    def incf(self,f,cat):
        '''Creates a feature/category pair if not exists, or increase the number
        if feature exists in a category'''
        if not self.usedb:
            self.fc.setdefault(f,{})
            self.fc[f].setdefault(cat,0)
            self.fc[f][cat] += 1
        else:
            count=self.fcount(f,cat)
            if count == 0:
                self.con.execute('INSERT INTO fc VALUES (?,?,?)',(f,cat,1))
            else:
                query = 'UPDATE fc SET COUNT=? WHERE feature=? AND category=?'
                self.con.execute(query,(count+1,f,cat)) 

    def incc(self,cat):
        '''Increases the number of occurrences of a category'''
        if not self.usedb:
            self.cc.setdefault(cat,0)
            self.cc[cat] += 1        
        else:
            count=self.catcount(cat)
            if count == 0:
                self.con.execute('INSERT INTO cc VALUES (?,?)',(cat,1))
            else:
                query = 'UPDATE cc SET count=? WHERE category=?'
                self.con.execute(query,(count+1,cat))    

    def catcount(self,cat):
        '''Counts the numer of itens in a category'''
        if not self.usedb:
            if cat in self.cc:
                return float(self.cc[cat])
            else:
                return 0
        else:
            query = "SELECT COUNT(*) FROM cc WHERE category=?"
            res=self.con.execute(query,(cat,)).fetchone()
            if res == None:
                return 0
            else:
                return float(res[0])

    def categories(self):
        '''Lists all the categories'''        
        if not self.usedb: return self.cc.keys()
        else:
            cur=self.con.execute('SELECT category FROM cc');
            return [d[0] for d in cur]

    def totalcount(self):
        ''' Returns the total numer of itens'''
        if not self.usedb: return sum(self.cc.values())
        else:
            res=self.con.execute('SELECT SUM(count) FROM cc').fetchone();
            if res==None: return 0
            else: return res[0]

    def train(self,item,cat):
        '''Receives an item (a bag of features) and a category, and increases
        the relative number of this category for all the features'''
        features=self.getfeatures(item)
        for f in features:
            self.incf(f,cat)
        self.incc(cat)
        if self.usedb: self.con.commit()

    def fprob(self,f,cat):
        '''Calculates the probability of a feature to be within a category'''
        if self.catcount(cat)== 0:
            return 0
        return self.fcount(f,cat)/float(self.catcount(cat))

    def weightedprob(self,f,cat,prf,weight=1.0,ap=0.5):
        '''Calculates the probability of a feature to appear in a certain category
        as fprob does, but assuming an initial value and changing according to 
        the training. That minimizes the effect of a rare word to be classified
        erroneously'''
        basicprob=prf(f,cat)
        totals=sum([self.fcount(f,c) for c in self.categories()])
        bp=((weight*ap)+(totals*basicprob))/(weight+totals)
        return bp

In [6]:
class naivebayes(classifier):
    '''Extends classifier class overriding __init__ and adding specific functions
    to classify documents using naive bayes'''
    
    def __init__(self, getfeatures, usedb=False):
        classifier.__init__(self,getfeatures)
        self.thresholds = {}
        self.usedb = usedb
        
    def docprob(self,item,cat):
        '''Calculates the probability of a document to be within a given
        category multiplying all the features probabilities to be in this category'''
        features=self.getfeatures(item)
        p=1
        for f in features: 
            p*=self.weightedprob(f,cat,self.fprob)
        return p

    def prob(self,item,cat):
        catprob=self.catcount(cat)/float(self.totalcount())
        docprob=self.docprob(item,cat)
        return docprob*catprob

    def setthreshold(self,cat,t):
        self.thresholds[cat]=t

    def getthreshold(self,cat):
        if cat not in self.thresholds: 
            return 1.0
        return self.thresholds[cat]

    def classify(self, item, default=None):
        '''Finds the most probably category to be set, and apply this
        classification, given that it satisfies a minimum threshold, compared
        to the second best category to classify; otherwise sets to "None"'''        
        probs = {}
        maximum = 0.0
        #best = None
        for cat in self.categories():
            probs[cat] = self.prob(item, cat)
            if probs[cat] > maximum: 
                maximum = probs[cat]
                best = cat
        for cat in probs:
            if cat == best:
                continue
            if probs[cat]*self.getthreshold(best) > probs[best]: 
                return default
        return best

In [7]:
class fisherclassifier(classifier):
    '''Extends classifier class overriding __init__ and adding specific functions
    to classify documents using fisher method'''

    def __init__(self,getfeatures, usedb=False):
        classifier.__init__(self, getfeatures)
        self.minimums = {}
        self.usedb = usedb
        
    def cprob(self,f,cat):
        '''Returns the frequency of the feature in a category divided
        by the frequency in all categories'''
        clf=self.fprob(f,cat)
        if clf == 0:
            return 0
        freqsum=sum([self.fprob(f,c) for c in self.categories()])
        p=float(clf)/(freqsum)
        return p

    def invchi2(self,chi, df):
        m = chi / 2.0
        sum = term = math.exp(-m)
        for i in range(1, df//2):
            term *= m / i
            sum += term
        return min(sum, 1.0)

    def prob(self,item,cat):
        '''Multipy all the categories, applies the natural log
        and uses the inverse chi2 to calculate probabilty'''
        p = 1
        features = self.getfeatures(item)
        for f in features:
            p *= (self.weightedprob(f,cat,self.cprob))
        fscore = -2*math.log(p)
        return self.invchi2(fscore,len(features)*2)

    def setminimum(self,cat, minimum):
        self.minimums[cat] = minimum

    def getminimum(self,cat):
        if cat not in self.minimums: return 0
        return self.minimums[cat]

    def classify(self, item, default=None):
        '''Applies fisher to all categories to find the best result, given 
        that it satisfies a minimum threshold, otherwise sets to "None"'''
        best = default
        maximum = 0.0
        for c in self.categories():
            p = self.prob(item,c)
            if p>self.getminimum(c) and p > maximum:
                best = c
                maximum = p
        return best

#### Third block of functions: reading files or searching for feeds

In [8]:
def blogread(url, classifier):
    '''Receives an url. Tries to classify the entries'''
    f = feedparser.parse(url)
    for entry in f['entries']:
        print('\n-----')
        if 'title' in entry:
            print('Title:     '+ entry['title'])
        if 'publisher' in entry:
            print('Publisher: '+ entry['publisher'])
        else:
            entry['publisher'] = 'Unknown'
        if 'updated_parsed' in entry:
            print('Date Published: ',datetime.datetime.fromtimestamp(time.mktime(entry['updated_parsed'])))
        print('\n-----')
        if 'summary' in entry:
            print(entry['summary'])
        guess = classifier.classify(entry)
        print('Suggested: {}'.format(guess))
        cl = input('Enter category or press <enter> to accept suggestion: ').lower()
        if cl == '':
            cl = guess
        if 'title' in entry:
            txt = 'Title:     '+entry['title']
        if 'publisher' in entry:
            txt = txt+'\n'+'Publisher: '+entry['publisher']
        if 'summary' in entry:
            txt = txt+'\n'+entry['summary']
        txt = txt+'\n'+'Suggested: {}'.format(guess)
        if cl == ''.strip() and guess:
            cl = guess
        print('Category "{}" chosen'.format(cl))
        classifier.train(entry,cl)

#### Fourth block of functions: instantiating and training classifiers

In [9]:
def sampletrain(cl):
    print('Running sampletrain to train the classifier...')
    cl.train('Nobody owns the water.','good')
    cl.train('the quick rabbit jumps fences','good')
    cl.train('buy pharmaceuticals now','bad')
    cl.train('make quick money at the online casino','bad')
    cl.train('the quick brown fox jumps','good')

In [10]:
def probabilidades_palavras():
    cl = classifier(getwords)
    print('\n')    
    sampletrain(cl)
    
    print('How many times "quick" --> "good": {}'.format(cl.fcount('quick','good')))
    print('How many times "quick" --> "bad": {}'.format(cl.fcount('quick','bad')))
    print('\nProbability of "quick" given that "good": {}'.format(cl.fprob('quick','good')))
    print('Probability of "money" given that "good" (fprob): {}'.format(cl.fprob('money','good')))
    print('Weighted probability of "money" given that "good" (weightedprob): {}'.format(cl.weightedprob('money','good',cl.fprob)))

    print('\nTraining again with the same documents...\n')
    sampletrain(cl)

    print('\nProbability of "money" given that "good" (fprob): {}'.format(cl.fprob('money','good')))
    print('Weighted probability of "money" given that "good" (weightedprob): {}\n'.format(cl.weightedprob('money','good',cl.fprob)))

In [11]:
probabilidades_palavras()



Running sampletrain to train the classifier...
How many times "quick" --> "good": 2.0
How many times "quick" --> "bad": 1.0

Probability of "quick" given that "good": 0.6666666666666666
Probability of "money" given that "good" (fprob): 0.0
Weighted probability of "money" given that "good" (weightedprob): 0.25

Training again with the same documents...

Running sampletrain to train the classifier...

Probability of "money" given that "good" (fprob): 0.0
Weighted probability of "money" given that "good" (weightedprob): 0.16666666666666666



In [12]:
def probabilidades_documentos_bayes():
    cl = naivebayes(getwords)
    print('\n')    
    sampletrain(cl)
    
    print('Classifying "quick rabbit": {}'.format(cl.classify('quick rabbit', default='unknown')))
    print('Classifying "quick money": {}'.format(cl.classify('quick money', default='unknown')))
    
    print('\nSetting the threshold up...')
    cl.setthreshold('bad',3.0)

    print('Classifying "quick money": {}'.format(cl.classify('quick money', default='unknown')))
    
    print('\nTraining again with the same documents (10x)...')
    for i in range(10): sampletrain(cl)
    
    print('\nClassifying "quick money": {}'.format(cl.classify('quick money', default='unknown')))

In [13]:
probabilidades_documentos_bayes()



Running sampletrain to train the classifier...
Classifying "quick rabbit": good
Classifying "quick money": bad

Setting the threshold up...
Classifying "quick money": unknown

Training again with the same documents (10x)...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...
Running sampletrain to train the classifier...

Classifying "quick money": bad


In [14]:
def probabilidades_palavras_fisher():
    cl = fisherclassifier(getwords)
    print('\n')    
    sampletrain(cl)
    print('\n')      
    print('Probability of "quick" given that "good": {}'.format(cl.cprob('quick', 'good')))
    print('Probability of "money" given that "bad": {}'.format(cl.cprob('money', 'bad')))
    print('Weighted probability of  "money" given that "bad": {}'.format(cl.weightedprob('money','bad',cl.cprob)))

In [15]:
probabilidades_palavras_fisher()



Running sampletrain to train the classifier...


Probability of "quick" given that "good": 0.5714285714285715
Probability of "money" given that "bad": 1.0
Weighted probability of  "money" given that "bad": 0.75


In [16]:
def probabilidades_documentos_fisher():
    cl = fisherclassifier(getwords)
    print('\n')    
    sampletrain(cl)

    print('Classifying "quick rabbit": {}'.format(cl.classify('quick rabbit')))
    print('Classifying "quick money": {}'.format(cl.classify('quick money')))
   
    print('\nSetting the threshold up...')
    cl.setminimum('bad',0.8)
    print('Classifying "quick money": {}'.format(cl.classify('quick money')))

    print('\nSetting the threshold down...')
    cl.setminimum('bad',0.4)
    print('Classifying "quick money": {}'.format(cl.classify('quick money')))

In [17]:
probabilidades_documentos_fisher()



Running sampletrain to train the classifier...
Classifying "quick rabbit": good
Classifying "quick money": bad

Setting the threshold up...
Classifying "quick money": good

Setting the threshold down...
Classifying "quick money": bad


In [18]:
def using_db_example():
    '''Training with a classifier, persisting in a database
    using the training data to classify using another classifier'''
    print('\nInstantiating a fisher classifier...')
    cl = fisherclassifier(getwords, usedb=True)
    cl.setdb(db_teste)
    sampletrain(cl)
    print('\nInstantiating a naive bayes classifier...')    
    cl2 = naivebayes(getwords, usedb=True)
    cl2.setdb(db_teste)
    print('Classifying "quick money": {}'.format(cl2.classify('quick money')))

In [19]:
using_db_example()


Instantiating a fisher classifier...
Running sampletrain to train the classifier...

Instantiating a naive bayes classifier...
Classifying "quick money": bad


In [20]:
def classifying_blogs(url):
    '''Instantiating a new classifier using "entryfeatures" (for feeds)
    Creating the database for the persistance of training data
    Using blogread with searching feeds option - no file reading'''
    print('\nList of categories stored in the database:')
    cl = fisherclassifier(entryfeatures, usedb=True)
    cl.setdb(db_blog)
    for category in cl.categories(): 
        print(category)
    blogread(url, cl) 
    print('\nList of categories stored in the database:')
    for category in cl.categories(): 
        print(category)
    return cl

In [21]:
url = 'http://g1.globo.com/dynamo/rss2.xml'

In [ ]:
cl = classifying_blogs(url)


List of categories stored in the database:

-----
Title:     Alunos são assaltados dentro de escola no bairro Vassoural, em Caruaru

-----
Quatro suspeitos pularam o muro da escola e foram até a quadra onde os estudantes estavam.  Dois estudantes foram assaltados na manhã desta terça-feira (29) na quadra da Escola de Referência em Ensino Médio Dom Miguel de Lima Valverde, no bairro Vassoural, em Caruaru, no Agreste de Pernambuco.
Segundo a direção da unidade de ensino, quatro suspeitos pularam o muro da escola e foram até a quadra onde os estudantes estavam. Os bandidos levaram o celular de um dos alunos.
O 1º Batalhão Integrado Especializado de Policiamento (BIESP) foi acionado e policiais conseguiram prender um dos assaltantes. De acordo com a polícia, ele havia fugido da Fundação de Atendimento Socioeducativo (Funase) no último sábado (26). Com ele foi apreendida uma faca. Os outros três suspeitos conseguiram fugir levando o celular.
Suggested: None


/usr/local/lib/python3.6/dist-packages/ipykernel/__main__.py:6: FutureWarning: split() requires a non-empty pattern match.
/usr/local/lib/python3.6/dist-packages/ipykernel/__main__.py:9: FutureWarning: split() requires a non-empty pattern match.


Category "policial" chosen

-----
Title:     Greve dos caminhoneiros: 9º dia

-----
<img src="https://s2.glbimg.com/DrGnA3fRty7C8wjh4qU2AX_Y9EM=/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2018/P/R/RDf3atS9ydnqlGo9EeSQ/protestos.jpg" /><br />   Protestos seguem pelo pais; assista à cobertura do Jornal Hoje: Greve dos caminhoneiros: 9º dia Protestos seguem pelo pais; assista à cobertura do Jornal Hoje: Abastecimento de combustíveis melhora, mas ainda é ruim em alguns estados. Há protestos em 24 estados e no DF; só Amapá e Amazonas não têm protestos. Falta combustível em 10 aeroportos; 66 vos foram cancelados, diz Infraero.  Ministro da Fazenda recua e descarta subir impostos para compensar diesel mais barato. Temer nega risco de intervenção militar e garante independência da Petrobras
Suggested: policial


Do some tests now:

In [28]:
#cl.cprob(<word>,<category>)
print(cl.cprob(u'corrupção','cultura'))

#cl.fprob(<word>,<category>)
print(cl.fprob('fields','ciencia'))

1.0
0
